# 12.7 参数服务器
- **目录**
  - 12.7.1 数据并行训练
  - 12.7.2 环同步
  - 12.7.3 多机训练

- 当将代码从一个GPU迁移到**多个GPU时**，以及再迁移到**包含多个GPU的多个服务器**时（可能所有服务器的分布跨越了多个机架和多个网络交换机），分布式并行训练算法也需要变得更加复杂。
- 通过细节可以知道：
  - 一方面是不同的互连方式的带宽存在极大的区别（例如，NVLink可以通过设置实现跨条链路的高达100GB/s的带宽，16通道的PCIe4.0提供32GB/s的带宽，而即使是高速100GbE以太网也只能提供大约10GB/s的带宽）；
  - 另一方面是期望开发者既能完成统计学习建模还精通系统和网络也是不切实际的。
- 后来又有人提出Push和Pull的语义，并描述了系统和开源库。
- 我们将介绍用于提高计算效率的组件。

## 12.7.1 数据并行训练

让我们回顾一下在分布式架构中数据并行的训练方法，因为在实践中它的实现相对简单，因此本节将排除其他内容只对其进行介绍。由于当今的GPU拥有大量的显存，因此在实际场景中（不包括图深度学习）**只有数据并行这种并行训练策略值得推荐**。图12.7.1描述了在 12.5节中实现的数据并行的变体。其中的关键是梯度的聚合需要在单个GPU（GPU 0）上完成，然后再将更新后的参数广播给所有GPU。

<center><img src='../img/ps.svg' height=400px width=400px></center>
<center>图12.7.1 左图是单GPU训练；右图是多GPU训练的一个变体：（1）计算损失和梯度，（2）所有梯度聚合在一个GPU上，（3）发生参数更新，并将参数重新广播给所有GPU</center><br>

回顾来看，选择GPU 0进行聚合似乎是个很随便的决定，当然也可以选择CPU上聚合，事实上只要优化算法支持，在实际操作中甚至可以在某个GPU上聚合其中一些参数，而在另一个GPU上聚合另一些参数。例如，如果有四个与参数向量相关的梯度$\mathbf{g}_1, \ldots, \mathbf{g}_4$，还可以一个GPU对一个$\mathbf{g}_i (i = 1, \ldots, 4$）进行梯度聚合。

这样的推断似乎是轻率和武断的，毕竟数学应该是逻辑自洽的。但是，我们处理的是如 12.4节中所述的真实的物理硬件，其中不同的总线具有不同的带宽。考虑一个如 12.4节中所述的真实的4路GPU服务器。如果它的连接是特别完整的，那么可能拥有一个100GbE的网卡。更有代表性的数字是1-10GbE范围内，其有效带宽为100MB/s到1GB/s。因为CPU的PCIe通道太少（例如，消费级的Intel CPU有24个通道），所以无法直接与所有的GPU相连接，因此需要multiplexer。CPU在16x Gen3链路上的带宽为16GB/s，这也是每个GPU连接到交换机的速度，这意味着GPU设备之间的通信更有效。

<center><img src='../img/bw-hierarchy.svg' height=400px width=400px></center>
<center>图12.7.2 一个4路GPU服务器</center><br>

为了便于讨论，我们假设所有梯度共需160MB。在这种情况下，将其中3个GPU的梯度发送到第4个GPU上需要30毫秒（每次传输需要10毫秒=160MB/16GB/s）。再加上30毫秒将权重向量传输回来，得到的结果是总共需要60毫秒。如果将所有的数据发送到CPU，总共需要80毫秒，其中将有40毫秒的惩罚，因为4个GPU每个都需要将数据发送到CPU。最后，假设能够将梯度分为4个部分，每个部分为40MB，现在可以在不同的GPU上同时聚合每个部分。因为PCIe交换机在所有链路之间提供全带宽操作，所以传输需要$2.5 \times 3 = 7.5$毫秒，而不是30毫秒，因此同步操作总共需要15毫秒。简而言之，一样的参数同步操作基于不同的策略时间可能在15毫秒到80毫秒之间。图12.7.3描述了交换参数的不同策略。

<center><img src='../img/ps-distributed.svg' ></center>
<center>图12.7.3 参数同步策略</center><br>

请注意，我们还可以使用另一个工具来改善性能：在深度网络中，从顶部到底部计算所有梯度需要一些时间，因此即使还在忙着为某些参数计算梯度时，就可以开始为准备好的参数同步梯度了。想了解如何操作可参考[Horovod](https://github.com/horovod/horovod)。

- **要点：**
  - **数据并行训练的方法**：
    - 数据并行训练是在分布式架构中的常用训练方法，因为它的实现相对简单。
    - 在现代GPU中，由于拥有大量显存，数据并行是推荐的并行训练策略。
  - **数据并行训练的过程**：
    - 计算损失和梯度。
    - 所有梯度聚合在一个GPU（例如GPU 0）上。
    - 发生参数更新，并将参数重新广播给所有GPU。
    - 可以在不同GPU上聚合不同的参数，例如在四个GPU上对四个梯度进行梯度聚合。
  - **通信效率问题**：
    - 在物理硬件中，不同的总线具有不同的带宽。
    - GPU与GPU之间的通信通常比GPU与CPU之间的通信更有效。
    - 参数同步的时间可能会根据不同的同步策略有很大差异。
  - **参数同步策略的选择**：
    - 将所有梯度发送到一个GPU上需要一定的时间。
    - 将所有数据发送到CPU可能更加耗时。
    - 将梯度分为多个部分，并在不同的GPU上同时聚合每个部分，可以减少传输时间。
  - **优化技巧**：
    - 可以在为某些参数计算梯度时就开始为准备好的参数同步梯度，以提高效率。
  - **可参考工具**：
    - Horovod是一个可以用于操作并改善性能的工具。

---
- **说明：图12.7.3 参数同步的三种架构解析**
  - 1. **CPU做参数服务器**
    - **架构描述**：
      - CPU作为参数服务器，负责存储和更新模型的参数。
      - 各GPU负责计算梯度，并将梯度传回CPU。
      - CPU更新参数后，将最新参数广播给各GPU。
    - **数据流**：
      - **步骤1**：每个GPU计算梯度，并将梯度传输给CPU。
      - **步骤2**：CPU用梯度更新参数。
      - **步骤3**：更新后的参数从CPU广播给所有GPU。
    - **优点**：
      - 结构简单，便于实现。
    - **缺点**：
      - CPU的计算能力和内存带宽成为瓶颈，限制性能。
      - 数据传输延迟较高。
  - 2. **GPU做参数服务器**
    - **架构描述**：
      - 参数服务器从CPU转移到GPU，由一个或多个GPU专门负责参数存储和更新。
      - 其他GPU仍然负责梯度计算。
      - 梯度和参数的传输在GPU之间完成，减少了CPU的参与。
    - **数据流**：
      - **步骤1**：每个计算GPU将计算好的梯度发送给负责参数更新的GPU。
      - **步骤2**：负责参数的GPU更新参数。
      - **步骤3**：更新后的参数从参数服务器GPU发送给各计算GPU。
    - **优点**：
      - 减少了CPU的负担，充分利用GPU的高速计算能力。
      - 数据传输延迟较低。
    - **缺点**：
      - 参数服务器GPU可能成为新的瓶颈。
      - 不适用于超大规模模型（单个GPU存储不足）。
  - 3. **参数服务器分布在所有GPU上**
    - **架构描述**：
      - 参数服务器分布在所有GPU上，每个GPU既负责计算梯度，也存储部分参数并更新。
      - 各GPU之间通过通信交换梯度和参数。
    - **数据流**：
      - **步骤1**：每个GPU计算梯度并发送给其他GPU。
      - **步骤2**：各GPU根据接收到的梯度更新参数。
      - **步骤3**：更新后的参数通过GPU间通信同步。
    - **优点**：
      - 最大化利用所有GPU资源，避免单点瓶颈。
      - 高效的计算和通信并行。
    - **缺点**：
      - 通信复杂度较高，可能出现网络瓶颈。
      - 需要高效的通信算法（如AllReduce）支持。
  - **对比分析**


| **架构**                  | **优点**                      | **缺点**                     | **适用场景**           |
| --------------------------- | ------------------------------- | ------------------------------ | ------------------------ |
| **CPU做参数服务器**       | 实现简单，硬件依赖低          | 通信开销大，CPU性能成为瓶颈  | 小规模模型或实验性开发 |
| **GPU做参数服务器**       | 减少CPU瓶颈，提升GPU利用率    | 参数服务器GPU可能成为瓶颈    | 中等规模模型训练       |
| **参数服务器分布在GPU上** | 充分利用GPU资源，避免单点瓶颈 | 通信复杂度高，需高效通信算法 | 大                     |

---

## 12.7.2 环同步（Ring Synchronization）

当谈及现代深度学习硬件的同步问题时，我们经常会遇到大量的定制的网络连接。例如，AWS p3.16xlarge和NVIDIA DGX-2实例中的连接都使用了  图12.7.4中的结构。每个GPU通过PCIe链路连接到主机CPU，该链路最多只能以16GB/s的速度运行。此外，每个GPU还具有$6$个NVLink连接，每个NVLink连接都能够以300Gbit/s进行双向传输。这相当于每个链路每个方向约$300\div 8\div 2\approx 18 \mathrm{GB/s}$。简言之，聚合的NVLink带宽明显高于PCIe带宽，问题是如何有效地使用它。

<center><img src='../img/nvlink.svg'></center>
<center>图12.7.4 在8台V100 GPU服务器上连接NVLink（图片由英伟达提供）</center><br>
研究结果表明最优的同步策略是将网络分解成两个环，并基于两个环直接同步数据。 图12.7.5描述了网络可以分解为一个具有双NVLink带宽的环（1-2-3-4-5-6-7-8-1）和一个具有常规带宽的环（1-4-6-3-5-8-2-7-1）。在这种情况下，设计一个高效的同步协议是非常重要的。

<center><img src='../img/nvlink-twoloop.svg'></center>
<center>图12.7.5 将NVLink网络分解为两个环</center><br>

考虑下面的思维试验：给定由$n$个计算节点（或GPU）组成的一个环，梯度可以从第一个节点发送到第二个节点，在第二个结点将本地的梯度与传送的梯度相加并发送到第三个节点，依此类推。在$n-1$步之后，可以在最后访问的节点中找到聚合梯度。也就是说，聚合梯度的时间随节点数线性增长。但如果照此操作，算法是相当低效的。归根结底，在任何时候都只有一个节点在通信。如果我们将梯度分为$n$个块，并从节点$i$开始同步块$i$，会怎么样？因为每个块的大小是$1/n$，所以总时间现在是$(n-1)/n \approx 1$。换句话说，当我们增大环的大小时，聚合梯度所花费的时间不会增加。这是一个相当惊人的结果。 图12.7.6说明了$n=4$个节点上的步骤顺序。


<center><img src='../img/ringsync.svg'></center>
<center>图12.7.6 跨4个节点的环同步。每个节点开始向其左邻居发送部分梯度，直到在其右邻居中找到聚合的梯度</center><br>

如果我们使用相同的例子，跨$8$个V100 GPU同步160MB，我们得到的结果大约是$2 \times 160 \mathrm{MB} \div (3 \times18 \mathrm{GB/s}) \approx 6 \mathrm{ms}$。这比使用PCIe总线要好，即使我们现在使用的是$8$个GPU。请注意，这些数字在实践中通常会差一些，因为深度学习框架无法将通信组合成大的突发传输。

注意到有一种常见的误解认为环同步与其他同步算法在本质上是不同的，实际上与简单的树算法相比其唯一的区别是同步路径稍微精细一些。

- **要点：**
  - **环同步的背景**：
    - 在现代深度学习硬件中，大量定制的网络连接经常出现。
    - 例如，AWS p3.16xlarge和NVIDIA DGX-2等实例中的连接通常通过NVLink和PCIe来完成，其中NVLink带宽明显高于PCIe带宽。
  - **NVLink连接**：
    - 每个GPU通过PCIe链路连接到主机CPU，速度最高为16GB/s。
    - 每个GPU还具有6个NVLink连接，每个连接都能进行约18GB/s的双向传输。
  - **最优同步策略**：
    - 研究表明，将网络分解为两个环并基于这两个环直接同步数据是最优的同步策略。
  - **环同步的思维试验**：
    - 考虑一个由$n$个计算节点组成的环，梯度可以沿环传递并相加。
    - 如果将梯度分为$n$个块，并从节点$i$开始同步块$i$，聚合梯度所花费的时间不会随着环的大小增加而增加。
    - 这意味着可以在多个节点之间高效同步。
  - **跨8个GPU的同步示例**：
    - 跨8个V100 GPU同步160MB的数据，使用环同步的时间大约是6毫秒。
    - 这比使用PCIe总线更好，即使使用更多的GPU。
  - **环同步与其他同步算法的比较**：
    - 有一种常见的误解，认为环同步与其他同步算法在本质上是不同的。
    - 事实上，与简单的树算法相比，环同步唯一的区别是同步路径稍微精细一些。
  - 环同步提供了一种有效利用现代深度学习硬件中的高带宽连接进行梯度聚合的方法，特别是在存在复杂网络连接和不同带宽的场景中。
  - 通过智能地组织通信，环同步确保了聚合梯度的时间不会随着节点数量的增加而增加，从而实现了高效的分布式训练。

## 12.7.3 多机训练

新的挑战出现在多台机器上进行分布式训练：我们需要服务器之间相互通信，而这些服务器又只通过相对较低的带宽结构连接，在某些情况下这种连接的速度可能会**慢一个数量级**，因此跨设备同步是个棘手的问题。毕竟，在不同机器上运行训练代码的速度会有细微的差别，因此如果想使用分布式优化的同步算法就需要**同步（synchronize）** 这些机器。
图12.7.7说明了分布式并行训练是如何发生的。

1. 在每台机器上读取一组（不同的）批量数据，在多个GPU之间分割数据并传输到GPU的显存中。基于每个GPU上的批量数据分别计算预测和梯度。
2. 来自一台机器上的所有的本地GPU的梯度聚合在一个GPU上（或者在不同的GPU上聚合梯度的某些部分）。
3. 每台机器的梯度被发送到其本地CPU中。
4. 所有的CPU将梯度发送到中央参数服务器中，由该服务器聚合所有梯度。
5. 然后使用聚合后的梯度来更新参数，并将更新后的参数广播回各个CPU中。
6. 更新后的参数信息发送到本地一个（或多个）GPU中。
7. 所有GPU上的参数更新完成。

<center><img src='../img/ps-multimachine.svg' width=700px height=700px/></center>
<center>图12.7.7 多机多GPU分布式并行训练</center><br>
以上这些操作似乎都相当简单，而且事实上它们可以在一台机器内高效地执行，但是当我们考虑多台机器时，就会发现<b>中央的参数服务器成为了瓶颈</b> 。毕竟，每个服务器的带宽是有限的，因此对于$m$个工作节点来说，将所有梯度发送到服务器所需的时间是$\mathcal{O}(m)$。我们也可以通过将参数服务器数量增加到$n$来突破这一障碍。此时，每个服务器只需要存储$\mathcal{O}(1/n)$个参数，因此更新和优化的总时间变为$\mathcal{O}(m/n)$。这两个数字的匹配会产生稳定的伸缩性，而不用在乎我们需要处理多少工作节点。在实际应用中，我们使用同一台机器既作为工作节点还作为服务器。设计说明请参考 图12.7.8。特别是，确保多台机器只在没有不合理延迟的情况下工作是相当困难的。我们在下面忽略了关于阻塞的细节，只简单介绍一下同步和异步（unsynchronized）更新。


<center><img src='../img/ps-multips.svg'/></center>
<center>图12.7.8 上图：单参数服务器是一个瓶颈，因为它的带宽是有限的；下图：多参数服务器使用聚合带宽存储部分参数</center><br>

- **要点：**
  - **多机训练的挑战**：
    - 服务器之间的通信通常通过较低的带宽结构连接，可能慢一个数量级。
    - 由于在不同机器上运行训练代码的速度可能有细微差别，跨设备同步成为一个棘手问题。
  - **分布式并行训练的过程**：
    - **数据读取和分割**：在每台机器上读取一组不同的批量数据，并在多个GPU间分割和传输。
    - **梯度计算和本地聚合**：在每个GPU上计算预测和梯度，并在一台机器的本地GPU中进行聚合。
    - **梯度传输到CPU**：每台机器的梯度被发送到本地CPU。
    - **中央聚合**：所有CPU将梯度发送到中央参数服务器，由该服务器聚合所有梯度。
    - **参数更新和广播**：使用聚合后的梯度更新参数，并将更新后的参数广播回各个CPU。
    - **参数信息传输**：更新后的参数信息发送到本地一个或多个GPU。
    - **GPU参数更新**：所有GPU上的参数更新完成。
  - **中央参数服务器的瓶颈**：
    - 中央参数服务器可能成为瓶颈，因为每个服务器的带宽是有限的。
    - 对于$m$个工作节点来说，将所有梯度发送到服务器所需的时间是$\mathcal{O}(m)$。
    - 通过增加参数服务器数量到$n$，使每个服务器只存储$\mathcal{O}(1/n)$个参数，可以突破这一障碍，总时间变为$\mathcal{O}(m/n)$。
  - **多参数服务器策略**：
    - 使用多参数服务器可以利用聚合带宽存储部分参数，以解决单参数服务器的带宽限制问题。
    - 在实际应用中，同一台机器可能既作为工作节点还作为服务器。
  - **同步和异步更新**：
    - 确保多台机器只在没有不合理延迟的情况下工作是困难的。
    - 同步和异步更新是两种主要的更新策略。

## 12.7.4 键值存储

在实践中，实现分布式多GPU训练所需要的步骤绝非易事。这就是**公共抽象**值得使用的原因，公共抽象即重新定义具有更新语义的**键－值存储（key-value store）** 的抽象。

在许多工作节点和许多GPU中，梯度$i$的计算可以定义为

$$\mathbf{g}_{i} = \sum_{k \in \text{workers}} \sum_{j \in \text{GPUs}} \mathbf{g}_{ijk} \tag{12.7.1}$$

其中$\mathbf{g}_{ijk}$是在工作节点$k$的GPU$j$上拆分的梯度$i$的一部分。这个运算的关键在于它是一个**交换归约（commutative reduction）**，也就是说，它把许多向量变换成一个向量，而运算顺序在完成向量变换时并不重要。这对实现我们的目标来说是非常好的，因为不需要为何时接收哪个梯度进行细粒度的控制。此外，请注意，这个操作在不同的$i$之间是独立的。

这就允许我们定义下面两个操作：**push**（用于累积梯度）和**pull**（用于取得聚合梯度）。因为我们有很多层，也就有很多不同的梯度集合，因此需要用一个键$i$来对梯度建索引。这个与Dynamo引入的“键－值存储”之间存在相似性并非巧合。它们两个定义都拥有许多相似的性质，特别是在多个服务器之间分发参数时。

“键－值存储”的push与pull操作描述如下：

* **push（key，value）** 将特定的梯度值从工作节点发送到公共存储，在那里通过某种方式（例如，相加）来聚合值。
* **pull（key，value）** 从公共存储中取得某种方式（例如，组合来自所有工作节点的梯度）的聚合值。

通过将同步的所有复杂性隐藏在一个简单的push和pull操作背后，我们可以将统计建模人员（他们希望能够用简单的术语表达优化）和系统工程师（他们需要处理分布式同步中固有的复杂性）的关注点解耦。

- **要点：**
  - **梯度计算的定义**：
    - 在多个工作节点和多个GPU中，梯度$i$的计算定义为$\mathbf{g}_{i} = \sum_{k \in \text{workers}} \sum_{j \in \text{GPUs}} \mathbf{g}_{ijk}$，其中$\mathbf{g}_{ijk}$是在工作节点$k$的GPU$j$上拆分的梯度$i$的一部分。
    - 这个运算是一个交换归约，意味着它把许多向量变换成一个向量，运算顺序并不重要，这对实现目标非常有利。
  - **键值存储操作**：
    - 这个概念允许定义两个操作：**push**（用于累积梯度）和**pull**（用于获取聚合梯度）。
    - 由于有许多层和不同的梯度集合，需要用键\(i\)对梯度进行索引。
  - **push和pull操作的描述**：
    - **push（key，value）**：将特定的梯度值从工作节点发送到公共存储，并以某种方式聚合值（例如相加）。
    - **pull（key，value）**：从公共存储中获取某种方式的聚合值（例如组合来自所有工作节点的梯度）。
  - **简化和解耦**：
    - 通过将同步的所有复杂性隐藏在push和pull操作背后，可以简化分布式训练的实现。
    - 这也允许将统计建模人员（关注优化）和系统工程师（处理分布式同步的复杂性）的关注点解耦。
  - **与Dynamo的相似性**：
    - 键值存储与Dynamo引入的“键－值存储”之间有相似性，特别是在多个服务器之间分发参数时。  

-------
- **说明：**
- **（1）何为公共抽象（Public Abstraction）？**
  - 公共抽象的概念：
    - 在分布式计算（尤其是多 GPU 训练）中，**公共抽象（Public Abstraction）** 指的是一种**通用的、高层次的接口或概念**。
    - 该技术隐藏了底层复杂的实现细节，使得不同角色（如统计建模人员和系统工程师）可以在更清晰的层次上协作。
  - 在本节讨论的**键-值存储（Key-Value Store）** 就是一个**公共抽象**，它将多 GPU 训练中的梯度同步操作封装为简单的 `push` 和 `pull` 操作，使得：
    - **统计建模人员** 只需关心如何优化模型，而不必关注底层的通信和同步机制。
    - **系统工程师** 只需实现高效的梯度同步，而不必关心具体的优化算法。
  - **为什么需要公共抽象？**
    - 在分布式训练中，多个计算节点（workers）和多个 GPU 需要**不断地通信和同步梯度**，但不同任务、不同系统的具体实现方式可能不同。直接操控底层通信会带来**复杂性、易错性和低效性**。
    - 因此需要**一个更高层的抽象**，让用户可以用简单的方式进行操作，而底层系统负责高效执行。
  - **公共抽象的作用：**
    - **隐藏系统复杂性**：统计建模人员只需使用 `push` 和 `pull` 操作，而不需要关心底层的网络通信、数据分片、同步机制等细节。
    - **提高可扩展性**：由于 `push` 和 `pull` 操作是通用的，它们可以用于不同的计算架构（如 CPU、GPU、TPU）和不同的分布式系统（如 Parameter Server、Ring All-Reduce）。
    - **提高代码复用性**：由于 `push` 和 `pull` 操作可以适用于不同的优化方法（如 SGD、Adam），开发人员可以更轻松地移植代码到不同的训练环境。
  - **键-值存储作为公共抽象**。在分布式梯度计算中可以使用**键-值存储**来管理梯度同步：
    - **键（Key）：** 代表某个特定的梯度变量（比如模型的某一层的权重梯度）。
    - **值（Value）：** 代表该梯度变量的值，可能存储在多个计算节点上。
  - **梯度同步过程：**
    - **Push（key, value）**：
      - 每个计算节点（worker）计算局部梯度后，调用 `push()`，将其发送到公共存储（如 Parameter Server）。
      - 公共存储会累加所有 worker 发送的梯度，使其聚合。
    - **Pull（key, value）**：
      - 计算节点在更新模型参数前，调用 `pull()` 获取聚合后的梯度值。
      - 这样，每个计算节点都能获得全局一致的梯度，并进行参数更新。
      - **"键-值存储"** 的思想类似于**分布式数据库或缓存**（如 Redis、DynamoDB），提供了一种通用的方式来存储和同步梯度。
  - **公共抽象的实际应用**
    - **Parameter Server（参数服务器）架构**
      - `push()` 用于将梯度上传到参数服务器。
      - `pull()` 用于从参数服务器获取最新的模型参数。
      - 适用于大规模深度学习训练（如 Google DistBelief、MXNet）。
    - **Ring All-Reduce（环状梯度聚合）**
      - `push()` 代表 GPU 之间的局部梯度交换。
      - `pull()` 代表最终聚合梯度的获取。
      - 被 PyTorch DDP、Horovod 等广泛使用。
    - **Federated Learning（联邦学习）**
      - `push()` 代表本地设备上传训练信息。
      - `pull()` 代表服务器下发全局模型更新。
      - 适用于隐私保护的分布式机器学习（如 Google FedAvg）。

- **（2）Dynamo简介**
  - Dynamo是由亚马逊开发的一个高度可扩展的键-值（key-value）存储系统。
    - 它是为了满足大型分布式系统中的需要，特别是在面对高吞吐量和可扩展性需求时。
  - 以下是有关Dynamo的一些关键点：
    - **键-值存储**：Dynamo提供了一个基本的键-值数据模型，允许客户端存储、检索和管理对象数据。每个对象通过一个唯一的键来识别。
    - **可扩展性**：Dynamo的设计专注于可扩展性，能够通过添加更多的机器来线性增加系统的容量和吞吐量。
    - **高可用性**：Dynamo通过数据的复制和分区来提供高可用性，确保在某些服务器或数据中心失效的情况下仍能访问数据。
    - **最终一致性**：与强一致性模型不同，Dynamo采用了最终一致性模型，这意味着在没有进一步的更新的情况下，所有复制 eventually会达到一致状态。这允许更高的性能和可用性。
    - **灵活性**：Dynamo允许应用程序开发人员选择与特定操作关联的一致性和可用性权衡，这使得它可以适应各种不同的使用场景和需求。
    - **分区和复制**：为了支持高度可扩展和容错，Dynamo使用一致性哈希来分区数据，并在多个节点之间复制数据。
    - **多数据中心支持**：Dynamo可以运行在多个数据中心中，提供地理上的冗余和故障隔离。
--------

## 小结

* 同步需要高度适应特定的网络基础设施和服务器内的连接，这种适应会严重影响同步所需的时间。
* 环同步对于p3和DGX-2服务器是最佳的，而对于其他服务器则未必。
* 当添加多个参数服务器以增加带宽时，**分层同步策略**可以工作的很好。

---
- **说明：大语言模型LLM的并行机制**
  
   在大语言模型（LLM）训练过程中，由于模型参数庞大、计算量巨大，通常需要 **大量GPU并行** 运行。训练时会采用 **多种并行策略**，主要包括以下几种：
- **1. 数据并行（Data Parallel, DP）**
  - **核心思想**：
    - **数据划分**：将 **相同的模型参数** 复制到多个 GPU，每个 GPU 处理不同的 **数据批次（mini-batch）**。
    - **梯度聚合**：每个 GPU 计算出梯度后，通过 **全局通信（如 All-Reduce）** 汇总梯度，并更新所有 GPU 的模型参数。
  - **适用场景**：
    - **模型较小**，但 **数据量大** 时适用。
    - 计算开销主要由前向和反向传播决定，梯度通信是主要的同步开销。
  - **常见实现**：
    - PyTorch **DistributedDataParallel (DDP)**
    - TensorFlow **MirroredStrategy**
    - NVIDIA **NCCL** 进行梯度通信
  - **挑战**：
    - **梯度通信瓶颈**：随着 GPU 数量增加，All-Reduce 操作的通信开销会变大。
    - **显存占用大**：每个 GPU 需要存储完整的模型参数，限制了超大模型的训练。
- **2. 张量并行（Tensor Parallel, TP）**
  - **核心思想**：
    - **模型划分**：将 **单个层的权重矩阵** 拆分到多个 GPU，每个 GPU 仅计算部分张量运算，并通过 **跨 GPU 通信** 交换计算结果。
    - 适用于 **矩阵乘法（如 Transformer 中的 FFN、Attention 计算）**。
  - **适用场景**：
    - **模型参数超大**，单个 GPU **无法存储整个模型**，但计算仍然可以拆分到多个 GPU 进行。
    - **NVIDIA Megatron-LM** 使用张量并行来训练 GPT-3 级别的大型模型。
  - **挑战**：
    - **跨 GPU 通信开销大**（尤其是 Transformer 的 Self-Attention 计算）。
    - **适配复杂**：需要手动拆分张量运算，如 QKV 投影、MLP 层等。
  - **常见实现**：
    - Megatron-LM（NVIDIA）
    - DeepSpeed-ZeRO（部分支持）

- **3. 流水线并行（Pipeline Parallel, PP）**
  - **核心思想**：
    - **模型层级拆分**：将 **不同的 Transformer 层** 分配到不同的 GPU，每个 GPU 只计算部分层的前向和反向传播。
    - 训练时采用 **流水线调度**，不同 GPU 交替计算不同 mini-batch 的不同阶段，以提高 GPU 利用率。
  - **适用场景**：
    - **模型层数极深**（如 GPT-3 175B 以上），单个 GPU 无法存储整个模型。
    - **数据并行和张量并行仍不足以解决显存问题时**。
  - **挑战**：
    - **存在等待时间**（bubble），流水线并行并不能完美加速，尤其是 batch size 较小时。
    - **实现复杂**，需要手动拆分 Transformer 层，并调整批次调度策略。
  - **常见实现**：
    - PyTorch **torch.distributed.pipeline.sync.Pipe**
    - Megatron-LM **Pipeline Parallel**
    - DeepSpeed **Pipeline Parallelism**
- **4. ZeRO 并行（Zero Redundancy Optimizer, ZeRO）**
  - **核心思想**：
    - 主要优化 **数据并行的显存占用问题**，通过拆分模型状态（如优化器状态、梯度、参数）到不同 GPU，减少冗余存储。
  - **ZeRO 主要有 3 个阶段（ZeRO-1 / ZeRO-2 / ZeRO-3）**：
    - **ZeRO-1**：仅拆分优化器状态，减少显存占用。
    - **ZeRO-2**：进一步拆分梯度存储，每个 GPU 仅存储部分梯度。
    - **ZeRO-3**：极限优化，连模型参数本身也进行分片，使得单个 GPU 仅需存储部分参数。
  - **适用场景**：
    - **超大模型（如 GPT-3 级别）**，即使使用张量并行和流水线并行，仍然需要 ZeRO 来减少显存消耗。
    - DeepSpeed 将 ZeRO 和其他并行策略（如数据并行、张量并行）结合，提升训练效率。
  - **挑战**：
    - **通信开销增大**，因为参数和梯度在多个 GPU 之间不断交换。
    - **调度复杂**，需要 DeepSpeed 进行高效的管理。
  - **常见实现**：
    - **DeepSpeed-ZeRO**
    - **Fairscale (ZeRO for PyTorch)**

- **5. 混合并行（Hybrid Parallelism）**
  - 在超大规模模型训练（如 GPT-3 / GPT-4 级别）时，**单一并行策略已经无法满足需求**，因此通常采用 **混合并行（Hybrid Parallelism）**：
    - **数据并行（DP） + 张量并行（TP）**：适用于中等规模的模型。
    - **数据并行（DP） + 流水线并行（PP）**：适用于深层模型。
    - **数据并行（DP） + 张量并行（TP） + 流水线并行（PP）**：适用于 GPT-3 以上的超大模型。
    - **数据并行（DP） + ZeRO 并行**：适用于减少显存占用的优化。
  - 例如：
    - **GPT-3（175B）** 训练时使用 **数据并行 + 张量并行 + 流水线并行** 组合。
    - **DeepSpeed** 结合 **ZeRO** 和 **其他并行策略** 进行高效训练。
- **总结**
| 并行策略             | 主要思想                     | 适用场景                     | 主要挑战        |
| ---------------------- | ------------------------------ | ------------------------------ | ----------------- |
| **数据并行（DP）**   | 复制模型，划分数据，梯度聚合 | 适用于小模型、大数据         | 梯度通信开销    |
| **张量并行（TP）**   | 拆分单层权重，跨 GPU 计算    | 适用于超大参数模型           | 通信复杂度高    |
| **流水线并行（PP）** | 拆分模型层，流水线调度       | 适用于超深模型               | 需要优化 bubble |
| **ZeRO 并行**        | 分片存储优化器/梯度/参数     | 适用于超大模型，降低显存占用 | 通信开销大      |
| **混合并行**         | 结合多种并行策略             | 适用于 GPT-3 级别超大模型    | 训练调度复杂    |

---